In [1]:
#| default_exp core

In [2]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [12]:
#| export
import scipy.sparse as sp, numpy as np, pandas as pd
from tqdm.auto import tqdm
from typing import Dict, Optional, List

In [8]:
#| export
def load_raw_txt(fname:str, encoding:str='utf-8', sep:Optional[str]='->'):
    ids, raw = [], []
    with open(fname, 'r', encoding=encoding) as file:
        for line in file:
            k, v = line[:-1].split(sep, maxsplit=1)
            ids.append(k); raw.append(v)
    return ids, raw

def load_raw_csv(fname:str, id_name:str="identifier", raw_name:str="text"):
    df = pd.read_csv(fname)
    df.fillna('', inplace=True)
    ids, raw = df[id_name].tolist(), df[raw_name].tolist()
    return ids, raw

def load_raw_file(fname:str, id_name:Optional[str]="identifier", raw_name:Optional[str]="text", 
                  sep:Optional[str]='->', encoding:Optional[str]='utf-8'):
    if fname.endswith(".txt"): 
        return load_raw_txt(fname, encoding=encoding, sep=sep)
    elif fname.endswith(".csv"): 
        return load_raw_csv(fname, id_name=id_name, raw_name=raw_name)
    else: 
        raise ValueError(f"Invalid filename: {fname}.")
    

In [ ]:
#| export
def get_all_ids(raw_dir, encoding='utf-8'):
    trn_ids, _ = load_raw_txt(f'{raw_dir}/train.raw.txt', encoding=encoding)
    tst_ids, _ = load_raw_txt(f'{raw_dir}/test.raw.txt', encoding=encoding)
    lbl_ids, _ = load_raw_txt(f'{raw_dir}/label.raw.txt', encoding=encoding)
    return set(trn_ids + tst_ids + lbl_ids)
    

In [ ]:
#| export
def filter_mapping(mapping, ids):
    return {id:mapping[id] for id in ids if id in mapping}
    

In [ ]:
#| export
def create_vocab_and_item2idx(mapping):
    mapping_item2idx, vocab = dict(), dict()
    for k,v in tqdm(mapping.items()):
        for o in v:
            idx = vocab.setdefault(o, len(vocab))
            l = mapping_item2idx.setdefault(k, [])
            l.append(idx)
    return vocab, mapping_item2idx
    

In [10]:
#| export
def save_raw_txt(fname, ids, raw, sep:Optional[str]='->', encoding:Optional[str]='utf-8'):
    assert len(ids) == len(raw), "Number of identifiers and elements in raw text should be the same."
    with open(fname, 'w', encoding=encoding) as file:
        for i,r in zip(ids, raw):
            i = str(i).replace("\n", "").replace("\t", "").replace("->", "")
            r = r.replace("\n", "").replace("\t", "").replace("->", "")
            file.write(f'{i}{sep}{r}\n')
            
def save_raw_csv(fname, ids, raw, id_name:Optional[str]="identifier", raw_name:Optional[str]="text"):
    assert len(ids) == len(raw), "Number of identifiers and elements in raw text should be the same."
    raw = [o.replace("\n", " ").replace("\r", " ") for o in raw]
    df = pd.DataFrame({id_name: ids, raw_name: raw})
    df.to_csv(fname, index=False)

def save_raw_file(fname, ids, raw, id_name:Optional[str]="identifier", raw_name:Optional[str]="text", 
                  sep:Optional[str]='->', encoding:Optional[str]='utf-8'):
    if fname.endswith(".txt"): 
        save_raw_txt(fname, ids, raw, sep=sep, encoding=encoding)
    elif fname.endswith(".csv"): 
        save_raw_csv(fname, ids, raw, id_name=id_name, raw_name=raw_name)
    else: 
        raise ValueError(f"Invalid filename: {fname}.")
            

In [15]:
#| export
def save_llm_input(fname:str, items:List, sep:Optional[str]='->', encoding:Optional[str]='utf-8'):
    for o in items: assert len(items[0]) == len(o), "Number of entities in all the items should be equal."
    with open(fname, 'w', encoding=encoding) as file:
        for item in zip(*items):
            line = ""
            for i,o in enumerate(item):
                o = str(o).replace("\n", "").replace("\t", "").replace("->", "")
                if i == 0: line = o
                else: line += o
            file.write(line + '\n')
            

In [ ]:
#| export
def get_matrix_from_item2idx(mapping, vocab_size, ids=None):
    data, indices, indptr = [], [], [0]
    ids = list(mapping) if ids is None else ids
    for i in tqdm(ids):
        if i in mapping:
            item_idx = mapping[i]
            data.extend([1]*len(item_idx))
            indices.extend(item_idx)
        indptr.append(len(data))
    return sp.csr_matrix((data, indices, indptr), shape=(len(ids), vocab_size), dtype=np.int64), ids
    

In [ ]:
#| export
def get_matrix_from_mapping(mapping, ids=None):
    if ids is not None:
        mapping = filter_mapping(mapping, ids)
    vocab, mapping_item2idx = create_vocab_and_item2idx(mapping)
    matrix, ids = get_matrix_from_item2idx(mapping_item2idx, len(vocab))
    return matrix, ids, vocab
    